# **I used BeautifulSoup for scrapping the given page.** 


In [1]:
from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print('Done')

Done


# Link to the page is wiki_url

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
response = requests.get(wiki_url)

In [4]:
soup = BeautifulSoup(response.text , 'html.parser')

In [5]:
our_table = soup.find('table') 

In [6]:
ldf = pd.read_html(str(our_table))

In [7]:
df = pd.DataFrame(ldf[0])

# df is the data frame that I created using the given link.

In [8]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df.shape

(180, 3)

# Checking for Not assigned values in data frame for Borough and then removing them using for loop.

In [10]:
print (len(df[df['Borough'] == 'Not assigned']))

77


In [11]:
for ndx, val in enumerate(df['Borough']):
  if (val == 'Not assigned'):
    df.drop(ndx, axis=0, inplace=True)

In [12]:
df.shape

(103, 3)

In [13]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# All Borough have Neighbourhood

In [14]:
print (len(df[df['Neighbourhood'] == 'Not assigned']))

0


# Shape for part I

In [15]:
df.shape

(103, 3)

# Part II, adding lat and lon co-ordinates. I added using the given csv file as Geocorder pakage did not work for me!

In [16]:
from google.colab import files

In [17]:
uploaded = files.upload()

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates (1).csv


In [18]:
import io

In [19]:
lat_lan = pd.read_csv(io.StringIO(uploaded['Geospatial_Coordinates.csv'].decode('utf-8')))

In [20]:
lat_lan.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Here I converted both the dataframes into descending order.

In [21]:
lat_lan.sort_values('Postal Code', ascending=False, ignore_index=True, inplace=True)

In [22]:
df.sort_values('Postal Code', ascending=False, ignore_index=True, inplace=True)

In [23]:
lat_lan.head()

,Postal Code,Latitude,Longitude
0,M9W,43.706748,-79.594054
1,M9V,43.739416,-79.588437
2,M9R,43.688905,-79.554724
3,M9P,43.696319,-79.532242
4,M9N,43.706876,-79.518188


In [24]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M9W,Etobicoke,"Northwest, West Humber - Clairville"
1,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
2,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
3,M9P,Etobicoke,Westmount
4,M9N,York,Weston


In [25]:
df['Latitude'] = lat_lan['Latitude']
df['Longitude'] = lat_lan['Longitude']

In [26]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054
1,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
2,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9N,York,Weston,43.706876,-79.518188


In [27]:
df.shape

(103, 5)

In [28]:
df.tail()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
98,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
99,M1G,Scarborough,Woburn,43.770992,-79.216917
100,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
101,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
102,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353


In [29]:
df.shape

(103, 5)

# Creating a dataframe that has boroughs with Toronto in their names.

In [30]:
var1 = df['Borough'].str.find('Toronto')

In [31]:
for ndx, val in enumerate(df['Borough']):
  if (val == 'Not assigned'):
    df.drop(ndx, axis=0, inplace=True)

In [32]:
tr_br = df

In [33]:
tr_br.drop(tr_br.loc[tr_br['Borough'].str.find('Toronto') == -1].index, inplace=True)

In [34]:
tr_br.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
15,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
17,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
18,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
19,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
20,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763


# Explode command converts the elements of the column which are in list format into single elments while keeping the other column values same.(adding rows)

In [40]:
tr_br = tr_br.explode('Neighbourhood', ignore_index=True)

# Map of Toronto City showsing all Neighbourhoods

In [36]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(lat, lon))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [37]:
map_toronto = folium.Map(location=[lat, lon], zoom_start=10)

for lat, lng, label in zip(tr_br['Latitude'], tr_br['Longitude'], tr_br['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Credential for 4 square

In [38]:
CLIENT_ID = 'E1FRENHEC2ITVIJ2UT5BY05O2UI2PEQHKHVKUDRPKMU2NZMS'
CLIENT_SECRET = 'IOYSYQQQXN1EDRQ0BU0M42QN4HO11V1RHNGPWFCQJGEVWATJ'
VERSION = '20180605'
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E1FRENHEC2ITVIJ2UT5BY05O2UI2PEQHKHVKUDRPKMU2NZMS
CLIENT_SECRET:IOYSYQQQXN1EDRQ0BU0M42QN4HO11V1RHNGPWFCQJGEVWATJ


In [39]:
tr_br.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
3,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
4,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763


# Here I created a new dataframe that shows differnt (100) venues in each Neighbourhood within the radius of 500 mt

In [54]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
tr_br_venues = getNearbyVenues(names=tr_br['Neighbourhood'],
                                   latitudes=tr_br['Latitude'],
                                   longitudes=tr_br['Longitude']
                                  )

Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
Queen's Park, Ontario Provincial Government
Runnymede, Swansea
Parkdale, Roncesvalles
High Park, The Junction South
Brockton, Parkdale Village, Exhibition Place
Little Portugal, Trinity
Dufferin, Dovercourt Village
Christie
First Canadian Place, Underground city
Stn A PO Boxes
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Kensington Market, Chinatown, Grange Park
University of Toronto, Harbord
The Annex, North Midtown, Yorkville
Forest Hill North & West, Forest Hill Road Park
Roselawn
Commerce Court, Victoria Hotel
Toronto Dominion Centre, Design Exchange
Harbourfront East, Union Station, Toronto Islands
Richmond, Adelaide, King
Central Bay Street
Berczy Park
St. James Town
Garden District, Ryerson
Regent Park, Harbourfront
Church and Wellesley
St. James Town, Cabbagetown
Rosedale
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer P

In [57]:
tr_br_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Rorschach Brewing Co.,43.663483,-79.319824,Brewery
1,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Leslieville Farmers Market,43.664901,-79.319784,Farmers Market
2,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,The Sidekick,43.664484,-79.325162,Comic Shop
3,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Queen Margherita Pizza,43.664685,-79.324164,Pizza Place
4,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Chino Locos,43.664653,-79.325584,Burrito Place


# showing number of venues for each Neighbourhood

In [96]:
tr_br_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,38,38,38,38,38,38


In [74]:
print('There are {} uniques categories.'.format(len(tr_br_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [94]:
tr_br_onehot = pd.get_dummies(tr_br_venues[['Venue Category']])

tr_br_onehot['Neighbourhood'] = tr_br_venues['Neighbourhood'] 

fixed_columns = [tr_br_onehot.columns[-1]] + list(tr_br_onehot.columns[:-1])
tr_br_onehot = tr_br_onehot[fixed_columns]

tr_br_onehot.head()

,Neighbourhood,Venue Category_Adult Boutique,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,Venue Category_Art Gallery,Venue Category_Art Museum,Venue Category_Arts & Crafts Store,Venue Category_Asian Restaurant,Venue Category_Auto Workshop,Venue Category_BBQ Joint,Venue Category_Baby Store,Venue Category_Bagel Shop,Venue Category_Bakery,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Stadium,Venue Category_Basketball Stadium,Venue Category_Beach,Venue Category_Bed & Breakfast,Venue Category_Beer Bar,Venue Category_Beer Store,Venue Category_Belgian Restaurant,Venue Category_Bistro,Venue Category_Boat or Ferry,Venue Category_Bookstore,Venue Category_Boutique,Venue Category_Brazilian Restaurant,Venue Category_Breakfast Spot,Venue Category_Brewery,Venue Category_Bubble Tea Shop,Venue Category_Building,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Bus Line,Venue Category_Business Service,Venue Category_Butcher,Venue Category_Café,Venue Category_Cajun / Creole Restaurant,Venue Category_Candy Store,Venue Category_Caribbean Restaurant,Venue Category_Cheese Shop,Venue Category_Chinese Restaurant,Venue Category_Chocolate Shop,Venue Category_Church,Venue Category_Climbing Gym,Venue Category_Clothing Store,Venue Category_Cocktail Bar,Venue Category_Coffee Shop,Venue Category_College Arts Building,Venue Category_College Auditorium,Venue Category_College Cafeteria,Venue Category_College Gym,Venue Category_College Rec Center,Venue Category_Colombian Restaurant,Venue Category_Comfort Food Restaurant,Venue Category_Comic Shop,Venue Category_Concert Hall,Venue Category_Convenience Store,Venue Category_Cosmetics Shop,Venue Category_Coworking Space,Venue Category_Creperie,Venue Category_Cuban Restaurant,Venue Category_Cupcake Shop,Venue Category_Dance Studio,Venue Category_Deli / Bodega,Venue Category_Department Store,Venue Category_Dessert Shop,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Distribution Center,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store,Venue Category_Escape Room,Venue Category_Ethiopian Restaurant,Venue Category_Event Space,Venue Category_Falafel Restaurant,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Filipino Restaurant,Venue Category_Fish & Chips Shop,Venue Category_Fish Market,Venue Category_Flea Market,Venue Category_Food,Venue Category_Food & Drink Shop,Venue Category_Food Court,Venue Category_Food Truck,Venue Category_Fountain,Venue Category_French Restaurant,Venue Category_Fried Chicken Joint,Venue Category_Frozen Yogurt Shop,Venue Category_Fruit & Vegetable Store,Venue Category_Furniture / Home Store,Venue Category_Gaming Cafe,Venue Category_Garden,Venue Category_Garden Center,Venue Category_Gas Station,Venue Category_Gastropub,Venue Category_Gay Bar,Venue Category_General Entertainment,Venue Category_General Travel,Venue Category_German Restaurant,Venue Category_Gift Shop,Venue Category_Gluten-free Restaurant,Venue Category_Gourmet Shop,Venue Category_Greek Restaurant,Venue Category_Grocery Store,Venue Category_Gym,Venue Category_Gym / Fitness Center,Venue Category_Harbor / Marina,Venue Category_Health & Beauty Service,Venue Category_Health Food Store,Venue Category_Historic Site,Venue Category_History Museum,Venue Category_Hobby Shop,Venue Category_Hookah Bar,Venue Category_Hospital,Venue Category_Hot Dog Joint,Venue Category_Hotel,Venue Category_Hotel Bar,Venue Category_IT Services,Venue Category_Ice Cream Shop,Venue Category_Indian Restaurant,Venue Category_Indie Movie Theater,Venue Category_Intersection,Venue Category_Irish Pub,Venue Category_Italian Restaurant,Venue Category_Japanese Restaurant

In [76]:
tr_br_grouped = tr_br_onehot.groupby('Neighbourhood').mean().reset_index()

# Showing top 5 popular places

In [77]:
num_top_venues = 5

for hood in tr_br_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = tr_br_grouped[tr_br_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.10
1    Cocktail Bar  0.05
2        Beer Bar  0.03
3  Farmers Market  0.03
4      Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4         Stadium  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.12
1           Yoga Studio  0.06
2               Brewery  0.06
3  Gym / Fitness Center  0.06
4         Garden Center  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  I

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tr_br_grouped['Neighbourhood']

for ind in np.arange(tr_br_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tr_br_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Bakery,Restaurant,Cheese Shop,Beer Bar,Diner,Jazz Club
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Italian Restaurant,Intersection,Bar,Stadium,Bakery,Climbing Gym,Furniture / Home Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Thai Restaurant,Salad Place,Burger Joint,Miscellaneous Shop,Modern European Restaurant


# clustering the data

In [79]:
kclusters = 5

tr_br_grouped_clustering = tr_br_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tr_br_grouped_clustering)

kmeans.labels_[0:10]

array([4, 4, 4, 4, 4, 2, 4, 4, 4, 4], dtype=int32)

In [80]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tr_br_merged = tr_br

tr_br_merged = tr_br_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

tr_br_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,4,Light Rail Station,Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,College Auditorium
2,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,4,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Pub,Diner,Bookstore,School,Sandwich Place,Burrito Place
3,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,4,Breakfast Spot,Gift Shop,Dessert Shop,Eastern European Restaurant,Dog Run,Bar,Italian Restaurant,Movie Theater,Restaurant,Bookstore
4,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,4,Mexican Restaurant,Thai Restaurant,Café,Grocery Store,Flea Market,Fried Chicken Joint,Bar,Italian Restaurant,Speakeasy,Fast Food Restaurant


#showing clusterd data

In [86]:
map_clusters = folium.Map(location=[lat, lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tr_br_merged['Latitude'], tr_br_merged['Longitude'], tr_br_merged['Neighbourhood'], tr_br_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [88]:
tr_br_merged.loc[tr_br_merged['Cluster Labels'] == 0, tr_br_merged.columns[[1] + list(range(5, tr_br_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Central Toronto,0,Ice Cream Shop,Garden,Pool,Music Venue,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Donut Shop


In [90]:
tr_br_merged.loc[tr_br_merged['Cluster Labels'] == 1, tr_br_merged.columns[[1] + list(range(5, tr_br_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
30,Central Toronto,1,Restaurant,Park,Tennis Court,Trail,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop


In [91]:
tr_br_merged.loc[tr_br_merged['Cluster Labels'] == 2, tr_br_merged.columns[[1] + list(range(5, tr_br_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,2,Grocery Store,Café,Park,Candy Store,Baby Store,Italian Restaurant,Restaurant,Nightclub,Bank,Coffee Shop


In [92]:
tr_br_merged.loc[tr_br_merged['Cluster Labels'] == 3, tr_br_merged.columns[[1] + list(range(5, tr_br_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Central Toronto,3,Park,Bus Line,Swim School,Business Service,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [93]:
tr_br_merged.loc[tr_br_merged['Cluster Labels'] == 4, tr_br_merged.columns[[1] + list(range(5, tr_br_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,4,Light Rail Station,Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park
1,Downtown Toronto,4,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,College Auditorium
2,West Toronto,4,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Pub,Diner,Bookstore,School,Sandwich Place,Burrito Place
3,West Toronto,4,Breakfast Spot,Gift Shop,Dessert Shop,Eastern European Restaurant,Dog Run,Bar,Italian Restaurant,Movie Theater,Restaurant,Bookstore
4,West Toronto,4,Mexican Restaurant,Thai Restaurant,Café,Grocery Store,Flea Market,Fried Chicken Joint,Bar,Italian Restaurant,Speakeasy,Fast Food Restaurant
5,West Toronto,4,Café,Coffee Shop,Breakfast Spot,Italian Restaurant,Intersection,Bar,Stadium,Bakery,Climbing Gym,Furniture / Home Store
6,West Toronto,4,Bar,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Asian Restaurant,Men's Store,Restaurant,Yoga Studio,Beer Store
7,West Toronto,4,Pharmacy,Bakery,Music Venue,Brazilian Restaurant,Café,Middle Eastern Restaurant,Supermarket,Bar,Bank,Pool
9,Downtown Toronto,4,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Deli / Bodega,Seafood Restaurant,Steakhouse,Asian Restaurant
10,Downtown Toronto,4,Coffee Shop,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Restaurant,Café,Beer Bar,Cocktail Bar,Art Gallery,Gym


# cluster 0 1 2 3 can be termed as outliers if we compare them with cluster 4.